# Why Use LCEL
### LangChain Expression Language (LCEL)

Alejandro Ricciardi (Omegapy)  
created date: 01/10/2024   
[GitHub](https://github.com/Omegapy)  

Credit: [LangChain](https://python.langchain.com/docs/expression_language/) 

Projects Description:  
**LangChain** is a framework for developing applications powered by language models. It enables applications that:

**In this project:**  I compare using LCEL and using LangChain without LCEL.   
LCEL makes it easier to build complex chains from basic components, and supports out of the box functionality such as streaming, parallelism, and logging
<p></p>
<b style="font-size:15;">
⚠️ This project requires an OpenAi key.
</b>

**Why LCEL**

**LangChain Expression Language**, or LCEL, is a declarative way to easily compose chains together. LCEL was designed from day 1 to **support putting prototypes in production, with no code changes,** from the simplest “prompt + LLM” chain to the most complex chains (we’ve seen folks successfully run LCEL chains with 100s of steps in production). To highlight a few of the reasons you might want to use LCEL:

**Streaming support** When you build your chains with LCEL you get the best possible time-to-first-token (time elapsed until the first chunk of output comes out). For some chains this means eg. we stream tokens straight from an LLM to a streaming output parser, and you get back parsed, incremental chunks of output at the same rate as the LLM provider outputs the raw tokens.

**Async support** Any chain built with LCEL can be called both with the synchronous API (eg. in your Jupyter notebook while prototyping) as well as with the asynchronous API (eg. in a [LangServe](https://python.langchain.com/docs/langsmith) server). This enables using the same code for prototypes and in production, with great performance, and the ability to handle many concurrent requests in the same server.

**Optimized parallel execution** Whenever your LCEL chains have steps that can be executed in parallel (eg if you fetch documents from multiple retrievers) we automatically do it, both in the sync and the async interfaces, for the smallest possible latency.

**Retries and fallbacks** Configure retries and fallbacks for any part of your LCEL chain. This is a great way to make your chains more reliable at scale. We’re currently working on adding streaming support for retries/fallbacks, so you can get the added reliability without any latency cost.

**Access intermediate** results For more complex chains it’s often very useful to access the results of intermediate steps even before the final output is produced. This can be used to let end-users know something is happening, or even just to debug your chain. You can stream intermediate results, and it’s available on every [LangServe](https://python.langchain.com/docs/langsmith) server.

**Input and output schemas** Input and output schemas give every LCEL chain Pydantic and JSONSchema schemas inferred from the structure of your chain. This can be used for validation of inputs and outputs, and is an integral part of LangServe.

**Seamless LangSmith tracing integration** As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at every step. With LCEL, all steps are automatically logged to LangSmith for maximum observability and debuggability.

**Seamless LangServe deployment integration** Any chain created with LCEL can be easily deployed using [LangServe](https://python.langchain.com/docs/langsmith).


### Project map:
- [API Key](#api-key)
- [Basic Example Using LCEL](#basic-example-using-lcel)
- [Invoke](#invoke)
    - [With LCEL](#with-lcel-invoke)
    - [Without LCEL](#without-lcel-invoke)
- [Stream](#stream)
    - [With LCEL](#with-lcel-steam)
    - [Without LCEL](#without-lcel-steam)
- [Batch](#batch)
    - [With LCEL](#with-lcel-batch)
    - [Without LCEL](#without-lcel-batch)
- [Async](#batch)
    - [With LCEL](#with-lcel-async)
    - [Without LCEL](#without-lcel-async)
- [LLM instead of chat model](#llm-instead-of-chat-model)
    - [With LCEL](#with-lcel-llm-instead-of-chat-model)
    - [Without LCEL](#without-lcel-llm-instead-of-chat-model)
- [Different model provider](#different-model-provider)
    - [With LCEL](#with-lcel-different-model-provider)
    - [Without LCEL](#without-lcel-different-model-provider)
- [Runtime configurability](#runtime-configurability)
    - [With LCEL](#with-lcel-runtime-configurability)
    - [Without LCEL](#without-lcel-runtime-configurability)
- [Logging](#logging)
    - [With LCEL](#with-lcel-logging)
    - [Without LCEL](#without-lcel-logging)
- [Fallbacks](#fallbacks)
    - [With LCEL](#with-lcel-fallbacks)
    - [Without LCEL](#without-lcel-fallbacks)
- [Full code comparison](#full-code-comparison)
    - [With LCEL](#with-lcel-full-code-comparison)
    - [Without LCEL](#without-lcel-full-code-comparison)



#### API Key

In [47]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())
OPENAI_API_KEY = os.environ.get("OPEN_AI_KEY")

from IPython.display import HTML

###  Basic Example Using LCEL
It is recommended to read the LCEL Get started [Intro LangChain LCEL.ipynb](https://github.com/Omegapy/LLM-Frameworks-Tutorials/blob/main/LangChain%20Tutorials/Tutorials%20from%20Langchain/Intro%20LangChain%20LCEL.ipynb) section first.


In [48]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

[Project map](#project-map)

### Invoke

##### With LCEL (Invoke)

In [49]:
from langchain_core.runnables import RunnablePassthrough


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-3.5-turbo")
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

chain.invoke("ice cream")

'Why did the ice cream go to therapy?\n\nBecause it had too many sprinkles of anxiety!'

##### Without LCEL (Invoke)

In [50]:
from typing import List

import openai


prompt_template = "Tell me a short joke about {topic}"
client = openai.OpenAI()

def call_chat_model(messages: List[dict]) -> str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=messages,
    )
    return response.choices[0].message.content

def invoke_chain(topic: str) -> str:
    prompt_value = prompt_template.format(topic=topic)
    messages = [{"role": "user", "content": prompt_value}]
    return call_chat_model(messages)

invoke_chain("ice cream")

'Why did the ice cream go to therapy?\n\nBecause it had too many cold sprinkles!'

[Project map](#project-map)

## Stream
Stream breacks up the AImessage in sentenses (chucks).

##### With LCEL (Steam)

In [51]:
for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)

Why did the ice cream go to therapy?
Because it had too many sprinkles of anxiety!

##### Without LCEL (Steam)

In [52]:
from typing import Iterator


def stream_chat_model(messages: List[dict]) -> Iterator[str]:
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        stream=True,
    )
    for response in stream:
        content = response.choices[0].delta.content
        if content is not None:
            yield content

def stream_chain(topic: str) -> Iterator[str]:
    prompt_value = prompt.format(topic=topic)
    return stream_chat_model([{"role": "user", "content": prompt_value}])


for chunk in stream_chain("ice cream"):
    print(chunk, end="", flush=True)

Sure, here's a joke for you: Why did the ice cream go to therapy? Because it had too many sprinkles of anxiety!

[Project map](#project-map)

### Batch
Running a batch of inputs in parallel 

##### With LCEL (Batch)

In [53]:
chain.batch(["ice cream", "spaghetti", "dumplings"]) # Note the three diferent prompt inputes

["Why did the ice cream go to therapy?\n\nBecause it had too many toppings and couldn't handle the sprinkles!",
 'Why did the tomato turn red?\n\nBecause it saw the spaghetti sauce!',
 'Why did the dumpling go to the party?\n\nBecause it wanted to get stuffed!']

Without LCEL (Batch)

In [54]:
from concurrent.futures import ThreadPoolExecutor

def batch_chain(topics: list) -> list:
    with ThreadPoolExecutor(max_workers=5) as executor:
        return list(executor.map(invoke_chain, topics))

batch_chain(["ice cream", "spaghetti", "dumplings"])

['Why did the ice cream go to therapy?\nBecause it always felt a little meltdramatic!',
 'Why did the tomato turn red?\n\nBecause it saw the spaghetti sauce!',
 'Why did the dumpling go to therapy?\n\nBecause it had too many emotional fillings!']

[Project map](#project-map)

### Async

Runs chains asynchronously

#### With LCEL (Async)

In [55]:
await chain.ainvoke("ice cream")

'Why did the ice cream go to the party? \nBecause it was feeling a little "meltdown"!'

#### Without LCEL (Async)

In [56]:
async_client = openai.AsyncOpenAI()

async def acall_chat_model(messages: List[dict]) -> str:
    response = await async_client.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=messages,
    )
    return response.choices[0].message.content

async def ainvoke_chain(topic: str) -> str:
    prompt_value = prompt_template.format(topic=topic)
    messages = [{"role": "user", "content": prompt_value}]
    return await acall_chat_model(messages)

await ainvoke_chain("ice cream")

'Why did the ice cream go to therapy?\n\nBecause it had too many emotional sundaes!'

[Project map](#project-map)

### LLM instead of chat model

#### With LCEL (LLM instead of chat model)

In [58]:
from langchain_openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | llm
    | output_parser
)

llm_chain.invoke("ice cream")

'\n\nWhy did the ice cream go to therapy?\n\nBecause it was feeling a little melon-choly.'

#### Without LCEL (LLM instead of chat model)

In [59]:
def call_llm(prompt_value: str) -> str:
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt_value,
    )
    return response.choices[0].text

def invoke_llm_chain(topic: str) -> str:
    prompt_value = prompt_template.format(topic=topic)
    return call_llm(prompt_value)

invoke_llm_chain("ice cream")

"\n\nWhy couldn't the bicycle ride to get ice cream?\n\nBecause it was two"

[Project map](#project-map)

### Different model provider

Anthropic instead of OpenAI
Note: this is just an example I do not have an Anthropic API Key -> code has no ouput

#### With LCEL (Different model provider)

In [ ]:
from langchain_community.chat_models import ChatAnthropic

anthropic = ChatAnthropic(model="claude-2")
anthropic_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | anthropic
    | output_parser
)

anthropic_chain.invoke("ice cream")

#### Without LCEL (Different model provider)

In [ ]:
import anthropic

anthropic_template = f"Human:\n\n{prompt_template}\n\nAssistant:"
anthropic_client = anthropic.Anthropic()

def call_anthropic(prompt_value: str) -> str:
    response = anthropic_client.completions.create(
        model="claude-2",
        prompt=prompt_value,
        max_tokens_to_sample=256,
    )
    return response.completion    

def invoke_anthropic_chain(topic: str) -> str:
    prompt_value = anthropic_template.format(topic=topic)
    return call_anthropic(prompt_value)

invoke_anthropic_chain("ice cream")

[Project map](#project-map)

### Runtime configurability

If we wanted to make the choice of chat model or LLM configurable at runtime

#### With LCEL (Runtime configurability)

In [ ]:
import anthropic
from langchain_core.runnables import ConfigurableField

configurable_model = model.configurable_alternatives(
    ConfigurableField(id="model"), 
    default_key="chat_openai", 
    openai=llm,
    anthropic=anthropic,
)

configurable_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | configurable_model 
    | output_parser
)

configurable_chain.invoke(
    "ice cream", 
    config={"model": "openai"}
)
stream = configurable_chain.stream(
    "ice cream", 
    config={"model": "anthropic"}
)
for chunk in stream:
    print(chunk, end="", flush=True)

configurable_chain.batch(["ice cream", "spaghetti", "dumplings"])

await configurable_chain.ainvoke("ice cream")

#### Without LCEL (Runtime configurability)

In [ ]:
def invoke_configurable_chain(
    topic: str, 
    *, 
    model: str = "chat_openai"
) -> str:
    if model == "chat_openai":
        return invoke_chain(topic)
    elif model == "openai":
        return invoke_llm_chain(topic)
    elif model == "anthropic":
        return invoke_anthropic_chain(topic)
    else:
        raise ValueError(
            f"Received invalid model '{model}'."
            " Expected one of chat_openai, openai, anthropic"
        )

def stream_configurable_chain(
    topic: str, 
    *, 
    model: str = "chat_openai"
) -> Iterator[str]:
    if model == "chat_openai":
        return stream_chain(topic)
    elif model == "openai":
        # Note we haven't implemented this yet.
        return stream_llm_chain(topic)
    elif model == "anthropic":
        # Note we haven't implemented this yet
        return stream_anthropic_chain(topic)
    else:
        raise ValueError(
            f"Received invalid model '{model}'."
            " Expected one of chat_openai, openai, anthropic"
        )

def batch_configurable_chain(
    topics: List[str], 
    *, 
    model: str = "chat_openai"
) -> List[str]:
    # You get the idea
    ...

async def abatch_configurable_chain(
    topics: List[str], 
    *, 
    model: str = "chat_openai"
) -> List[str]:
    ...

invoke_configurable_chain("ice cream", model="openai")
stream = stream_configurable_chain(
    "ice_cream", 
    model="anthropic"
)
for chunk in stream:
    print(chunk, end="", flush=True)

# batch_configurable_chain(["ice cream", "spaghetti", "dumplings"])
# await ainvoke_configurable_chain("ice cream")

[Project map](#project-map)

### Logging

Logging into LangChain using a LangChain API Key.

#### With LCEL (Logging)

Note: I do note have a LangChain API Key (https://smith.langchain.com/o/bd3fb686-7fb6-557c-95f9-8f1d8dcb7aee/)

In [ ]:
import os

os.environ["LANGCHAIN_API_KEY"] = "..."
os.environ["LANGCHAIN_TRACING_V2"] = "true"

anthropic_chain.invoke("ice cream")

#### Without LCEL (Logging)

In [ ]:
def invoke_anthropic_chain_with_logging(topic: str) -> str:
    print(f"Input: {topic}")
    prompt_value = anthropic_template.format(topic=topic)
    print(f"Formatted prompt: {prompt_value}")
    output = call_anthropic(prompt_value)
    print(f"Output: {output}")
    return output

invoke_anthropic_chain_with_logging("ice cream")

[Project map](#project-map)

### Fallbacks

If we wanted to add fallback logic, in case one model API is down
(https://python.langchain.com/docs/guides/fallbacks)

#### With LCEL (Fallbacks)

In [61]:
from langchain_openai import ChatOpenAI

chat_prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# bad model name to easily create a chain that will error
chat_model = ChatOpenAI(model_name="gpt-fake")
bad_chain = chat_prompt | chat_model | StrOutputParser()

# good chain
chat_openai = ChatOpenAI(model="gpt-4")
good_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | chat_openai
    | output_parser
)

# Note if bad_chain fails good chain will run
fallback_chain = bad_chain.with_fallbacks([good_chain])

# fallback_chain.invoke("ice cream")
# or
# await fallback_chain.ainvoke("ice cream")
# or
fallback_chain.batch(["ice cream", "spaghetti", "dumplings"])

["Why don't ice creams ever get invited to parties?\n\nBecause they always bring a meltdown!",
 'Why don’t we tell secrets on a farm?\n\nBecause the potatoes have eyes, the corn has ears, and the spaghetti is pasta-tively terrible at keeping secrets!',
 "Why don't dumplings ever win at hide and seek?\n\nBecause they always meet at the steamer!"]

#### Without LCEL (Fallbacks)

In [62]:

def invoke_chain_with_fallback(topic: str) -> str:
    try:
        return invoke_chain(topic)
    except Exception:
        return invoke_anthropic_chain(topic)

async def ainvoke_chain_with_fallback(topic: str) -> str:
    try:
        return await ainvoke_chain(topic)
    except Exception:
        # Note: we haven't actually implemented this.
        return ainvoke_anthropic_chain(topic)

async def batch_chain_with_fallback(topics: List[str]) -> str:
    try:
        return batch_chain(topics)
    except Exception:
        # Note: we haven't actually implemented this.
        return batch_anthropic_chain(topics)

#invoke_chain_with_fallback("ice cream")
# or
# await ainvoke_chain_with_fallback("ice cream")
# or
await batch_chain_with_fallback(["ice cream", "spaghetti", "dumplings"])

['Why did the ice cream go to the party? \nBecause it was looking to get "scooped" up and have a "cone" of a good time!',
 'Why did the spaghetti bring a ladder? \n\nBecause it wanted to "pasta" sauce!',
 'Why did the dumpling refuse to have dessert?\n\nBecause it was already stuffed!']

[Project map](#project-map)

### Full code comparison

You need an Anthropic and LangChain API key to run the full code.

#### With LCEL (Full code comparison)

In [ ]:
import os

from langchain_community.chat_models import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, ConfigurableField

os.environ["LANGCHAIN_API_KEY"] = "..."
os.environ["LANGCHAIN_TRACING_V2"] = "true"

prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
chat_openai = ChatOpenAI(model="gpt-3.5-turbo")
openai = OpenAI(model="gpt-3.5-turbo-instruct")
anthropic = ChatAnthropic(model="claude-2")
model = (
    chat_openai
    .with_fallbacks([anthropic])
    .configurable_alternatives(
        ConfigurableField(id="model"),
        default_key="chat_openai",
        openai=openai,
        anthropic=anthropic,
    )
)

chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

#### Without LCEL (Full code comparison)

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from typing import Iterator, List, Tuple

import anthropic
import openai


prompt_template = "Tell me a short joke about {topic}"
anthropic_template = f"Human:\n\n{prompt_template}\n\nAssistant:"
client = openai.OpenAI()
async_client = openai.AsyncOpenAI()
anthropic_client = anthropic.Anthropic()

def call_chat_model(messages: List[dict]) -> str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=messages,
    )
    return response.choices[0].message.content

def invoke_chain(topic: str) -> str:
    print(f"Input: {topic}")
    prompt_value = prompt_template.format(topic=topic)
    print(f"Formatted prompt: {prompt_value}")
    messages = [{"role": "user", "content": prompt_value}]
    output = call_chat_model(messages)
    print(f"Output: {output}")
    return output

def stream_chat_model(messages: List[dict]) -> Iterator[str]:
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        stream=True,
    )
    for response in stream:
        content = response.choices[0].delta.content
        if content is not None:
            yield content

def stream_chain(topic: str) -> Iterator[str]:
    print(f"Input: {topic}")
    prompt_value = prompt.format(topic=topic)
    print(f"Formatted prompt: {prompt_value}")
    stream = stream_chat_model([{"role": "user", "content": prompt_value}])
    for chunk in stream:
        print(f"Token: {chunk}", end="")
        yield chunk

def batch_chain(topics: list) -> list:
    with ThreadPoolExecutor(max_workers=5) as executor:
        return list(executor.map(invoke_chain, topics))

def call_llm(prompt_value: str) -> str:
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt_value,
    )
    return response.choices[0].text

def invoke_llm_chain(topic: str) -> str:
    print(f"Input: {topic}")
    prompt_value = promtp_template.format(topic=topic)
    print(f"Formatted prompt: {prompt_value}")
    output = call_llm(prompt_value)
    print(f"Output: {output}")
    return output

def call_anthropic(prompt_value: str) -> str:
    response = anthropic_client.completions.create(
        model="claude-2",
        prompt=prompt_value,
        max_tokens_to_sample=256,
    )
    return response.completion   

def invoke_anthropic_chain(topic: str) -> str:
    print(f"Input: {topic}")
    prompt_value = anthropic_template.format(topic=topic)
    print(f"Formatted prompt: {prompt_value}")
    output = call_anthropic(prompt_value)
    print(f"Output: {output}")
    return output

async def ainvoke_anthropic_chain(topic: str) -> str:
    ...

def stream_anthropic_chain(topic: str) -> Iterator[str]:
    ...

def batch_anthropic_chain(topics: List[str]) -> List[str]:
    ...

def invoke_configurable_chain(
    topic: str, 
    *, 
    model: str = "chat_openai"
) -> str:
    if model == "chat_openai":
        return invoke_chain(topic)
    elif model == "openai":
        return invoke_llm_chain(topic)
    elif model == "anthropic":
        return invoke_anthropic_chain(topic)
    else:
        raise ValueError(
            f"Received invalid model '{model}'."
            " Expected one of chat_openai, openai, anthropic"
        )

def stream_configurable_chain(
    topic: str, 
    *, 
    model: str = "chat_openai"
) -> Iterator[str]:
    if model == "chat_openai":
        return stream_chain(topic)
    elif model == "openai":
        # Note we haven't implemented this yet.
        return stream_llm_chain(topic)
    elif model == "anthropic":
        # Note we haven't implemented this yet
        return stream_anthropic_chain(topic)
    else:
        raise ValueError(
            f"Received invalid model '{model}'."
            " Expected one of chat_openai, openai, anthropic"
        )

def batch_configurable_chain(
    topics: List[str], 
    *, 
    model: str = "chat_openai"
) -> List[str]:
    ...

async def abatch_configurable_chain(
    topics: List[str], 
    *, 
    model: str = "chat_openai"
) -> List[str]:
    ...

def invoke_chain_with_fallback(topic: str) -> str:
    try:
        return invoke_chain(topic)
    except Exception:
        return invoke_anthropic_chain(topic)

async def ainvoke_chain_with_fallback(topic: str) -> str:
    try:
        return await ainvoke_chain(topic)
    except Exception:
        return ainvoke_anthropic_chain(topic)

async def batch_chain_with_fallback(topics: List[str]) -> str:
    try:
        return batch_chain(topics)
    except Exception:
        return batch_anthropic_chain(topics)

[Project map](#project-map)